# Logistic Regression Baseline

This notebook will build a basic logistic regression model and evaluate results using cross-validation.

1. Read in the data
2. Create hold-out test set
3. Create pipeline for logistic regression. 
    - Random Oversample of data to balance out target.
    - cross-validation of results; evaluation metrics are accuracy, f1 score, and AUC. 
  

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from data_import import preprocess_data
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split, cross_validate
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.pipeline import make_pipeline
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from Evaluation import eval_model

/Users/khickey/anaconda3/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
#load in the preprocessed data
data = preprocess_data('Statcast_data.csv')
target = data['description']
#filter out 'player_name'; will not use as a feature
data = data.iloc[:, :-2]

/Users/khickey/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
#first, create a test set
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state = 777, test_size = .2)

#then instatiate the model we will use: Logistic Regression
log_reg = LogisticRegression()


## Build Model and Interpret

Build a model and apply interpretation of model on the results. This involves analyzing the values of the coefficients.

Train on one fold of training data, and test on a validation set. We will randomely oversample to balance out the ratio of classes in the target

In [4]:
ros = RandomOverSampler(ratio = 1, random_state =777)

X_os, y_os = ros.fit_resample(X_train, y_train)

log_reg.fit(X_os, y_os)

y_pred_logreg = log_reg.predict(X_test)

print(f" Test Accuracy score: {accuracy_score(y_test, y_pred_logreg)}")
print(f" Test AUC score: {roc_auc_score(y_test, y_pred_logreg)}")
print(f" Test F1 score: {f1_score(y_test, y_pred_logreg)}")

 Test Accuracy score: 0.49475850287311696
 Test AUC score: 0.5438088316722038
 Test F1 score: 0.46335931378613554


In [5]:
for idx, feature in enumerate(X_train.columns):
    print(f"{feature} coefficient: {log_reg.coef_[0][idx]}")


release_speed coefficient: 0.06911968625592822
release_spin_rate coefficient: 0.00276223713076353
release_pos_x coefficient: 0.017508759277576218
release_pos_y coefficient: 0.48146282067779284
release_pos_z coefficient: -0.0011657378185506147
pfx_x coefficient: 0.08050853168377756
pfx_z coefficient: 0.13044940176037795
vx0 coefficient: -0.011622473367725728
vy0 coefficient: 0.12407318150772563
vz0 coefficient: 0.00202079165490292
ax coefficient: -0.10879181621088845
ay coefficient: -0.013328955716309578
az coefficient: -0.08995007570798766
sz_top coefficient: 0.0018661102717056513
sz_bot coefficient: -0.0003097677234898032
release_extension coefficient: 0.47920587517046037
pitch_name_2-Seam Fastball coefficient: 0.2516369510978865
pitch_name_4-Seam Fastball coefficient: 0.2763296571980953
pitch_name_Changeup coefficient: -0.49606918039456654
pitch_name_Curveball coefficient: 0.14274307306521972
pitch_name_Cutter coefficient: 0.05538028109718202
pitch_name_Sinker coefficient: 0.34973680

## Analysis

Looking at the coefficients, there is one main property that can be interpreted: the signage, which can ultimately help determine the odds ratio for each feature. Negative value coefficients result in a odds ratio below 1, meaning that one unit change in that feature will lower the chance of a positive target value, in this case a strike. A positive value will indicate produce a positive odds ratio, meaning that a one unit change in that feature will make it more likely of a resulting strike. This however, assumes holding all other features constant, which is highly impractical.

cite: http://www.appstate.edu/~whiteheadjc/service/logit/intro.htm#interp



## Cross-validation

#### Use Cross-validation to ensure reliability of results above.

Use custom function in `Evaluation.py` module.

In [6]:
eval_model(log_reg, X_train, y_train, 5)

Mean test_accuracy Value: 0.48894387316710847
test_accuracy scores: [0.48599854 0.49157971 0.49174917 0.48454109 0.49085085]



KeyError: 'train_accuracy'